# Imports

In [1]:
import time
time_start_notebook = time.time()

In [2]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path

from langchain.text_splitter import CharacterTextSplitter
import pypdf
from PyPDF2 import PdfReader
from pypdf import PdfWriter
from pdf2image import convert_from_path, convert_from_bytes
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter


from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

import streamlit
import streamlit as st
import base64
import os
import glob

import streamlit as st
from dotenv import load_dotenv
from werkzeug.utils import secure_filename
import os
import sys
import glob

import openai
import langchain
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain.llms import AzureOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

#--- ocr
import pdf2image
from pdf2image import convert_from_path, convert_from_bytes
import pytesseract

In [3]:
%load_ext watermark

In [4]:
%watermark -iv

langchain  : 0.1.16
streamlit  : 1.33.0
pandas     : 1.5.3
numpy      : 1.26.4
pytesseract: 0.3.10
pypdf      : 4.2.0
openai     : 1.22.0
sys        : 3.11.7 (tags/v3.11.7:fa7a6f2, Dec  4 2023, 19:24:49) [MSC v.1937 64 bit (AMD64)]
pdf2image  : 1.17.0



# Config

In [5]:
from pathlib import Path
path_pdf_ocr = r"C:\Users\a126291\OneDrive - AmerisourceBergen(ABC)\data\pdf_files\ocr_pdf\screenshot_pages_eylea.pdf"
path_pdf_non_ocr = r"C:\Users\a126291\OneDrive - AmerisourceBergen(ABC)\data\pdf_files\fda_drugs\fda_eylea.pdf"

# source directory

In [6]:
import shutil
if os.path.isdir('src'):
    shutil.rmtree('src')

In [7]:
import os
if not os.path.isdir('src'):
    os.makedirs('src')

In [8]:
with open('src/__init__.py','w') as fh:
    fh.write('# Author: Bhishan Poudel\n')

In [9]:
!ls src

__init__.py


In [10]:
!cat src/__init__.py

# Author: Bhishan Poudel


# util: config

In [11]:
%%writefile src/config.py

def get_config():
    # config
    openai_api_key = "f4001c4b49a845309d386e9014b7ae6b"
    openai_api_base = "https://qa.gai.cencora.com/aoai"
    azure_endpoint = openai_api_base
    openai_api_version = "2023-03-15-preview"
    openai_api_type = "azure"

    openai_deployment_name = "gpt-35-turbo-16k"
    openai_model_name      = "gpt-35-turbo-16k"

    # os environments
    import os
    os.environ['OPENAI_API_TYPE'] = openai_api_type
    os.environ['OPENAI_API_VERSION'] = openai_api_version
    os.environ['OPENAI_API_BASE'] = openai_api_base
    os.environ['OPENAI_API_KEY'] = openai_api_key

    # openai environments
    import openai
    openai.api_key = openai_api_key
    openai.api_type = openai_api_type
    openai.api_base = openai_api_base
    openai.api_version = openai_api_version
    openai.api_key = openai_api_key

    config = {
    'openai_api_base': openai_api_base,
    'openai_api_version': openai_api_version,
    'deployment_name': openai_deployment_name,
    'model_name': openai_model_name,
    'openai_api_key': openai_api_key,
    'openai_api_type': openai_api_type
    }

    return config

config = get_config()

Writing src/config.py


In [12]:
from src.config import config

config['model_name']

'gpt-35-turbo-16k'

# Util: ocr

In [13]:
%%writefile src/util_ocr.py
import streamlit as st
import os

#--- ocr
import pdf2image
import pytesseract
pytesseract.tesseract_cmd = r"C:\Users\a126291\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

#@st.cache_data
def get_ocr_lst_text_from_uploaded_file(uploaded_file, language='eng',pytesseract_path=None):
    import pdf2image
    import pytesseract
    from pathlib import Path

    if pytesseract_path == None:
        pytesseract_path = r"C:\Users\a126291\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
    pytesseract.tesseract_cmd = pytesseract_path

    try: # streamlit uploded file
        data_bytes = uploaded_file.getvalue()
    except:
        try: # regular file path
            path_pdf = Path(uploaded_file)
            if path_pdf.exists():
                with open(path_pdf, 'rb') as file:
                    data_bytes = file.read()
        except:
            pass

    pil_images = pdf2image.convert_from_bytes(data_bytes)
    lst_txt = []
    for pil_image in pil_images:
        text_one_page = pytesseract.image_to_string(pil_image, lang=language)
        lst_txt.append(text_one_page)
    return lst_txt

Writing src/util_ocr.py


In [14]:
from src.util_ocr import get_ocr_lst_text_from_uploaded_file

path_pdf = path_pdf_ocr
lst_text = get_ocr_lst_text_from_uploaded_file(uploaded_file=path_pdf, language='eng',pytesseract_path=None)

print(lst_text[0][-500:])

croliters) admuistered by
intravitreal injection every 4 weeks (monthly) for the frst 5 imjections, followed by 2 mg

(0.05 mL) via intravitreal injection once every 8 weeks (2 months). Although EYLEA may be
dosed as frequently as 2 mg every 4 weeks (monthly), additional efficacy was not demonstrated
10 most patients when EYLEA was dosed every 4 weeks compared to every 8 weeks [see
Clinical Studies (14.4)]. Some patients may need every 4 week (monthly) dosing after the firs
20 weeks (5 months).



# Util: Text

In [15]:
%%writefile src/util_text.py

#========================== util: text
from langchain.text_splitter import CharacterTextSplitter
from PyPDF2 import PdfReader
from pypdf import PdfWriter
from pdf2image import convert_from_path, convert_from_bytes
import os
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# local imports
from src.util_ocr import get_ocr_lst_text_from_uploaded_file

# import pytesseract
import pytesseract
pytesseract.tesseract_cmd = r"C:\Users\a126291\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

# Single PDF No OCR
def get_lst_text_from_single_pdf_no_ocr(path_pdf):
    r'Get list of text for each pages of pdf document'
    from PyPDF2 import PdfReader

    pdf_reader = PdfReader(path_pdf)
    lst_text = []
    for page in pdf_reader.pages:
        text = page.extract_text()
        lst_text.append(text)
    return lst_text


def get_lst_text_from_uploaded_file(uploaded_file,ocr=False,pytesseract_path=None):
    r'''Get list of text from a single pdf (works for both ocr and non-ocr pdf).

    '''
    lst_text = []
    if ocr:
        lst_text = get_ocr_lst_text_from_uploaded_file(uploaded_file, language='eng',pytesseract_path=None)
    else:
        lst_text = get_lst_text_from_single_pdf_no_ocr(uploaded_file)
    return lst_text


Writing src/util_text.py


In [16]:
# !explorer.exe .

In [17]:
from src.util_text import get_lst_text_from_uploaded_file
uploaded_file = path_pdf_ocr
lst_text = get_lst_text_from_uploaded_file(uploaded_file,ocr=True,pytesseract_path=None)

In [18]:
len(lst_text)

6

In [19]:
if lst_text:
    print(lst_text[0])

y DOSAGE AND ADMINISTRATION

21 Important Injection Instructions

For ophthalmic intravitreal injection. EYLEA must only be admumstered by a qualified
physician.

2.2 — Neovascular (Wet) Age-Related Macular Degeneration (AMD)

The recommended dose for EYLEA 1s 2 me (0.05 mL or 50 microliters) admuistered by
intravitreal injection every 4 weeks (monthly) for the first 12 weeks (3 months), followed by

2 me (0.05 mL) via intravitreal injection once every 8 weeks (2 months). Although EYLEA may
be dosed as frequently as 2 me every 4 weeks (monthly), additional efficacy was not
demonstrated in most patients when EYLEA was dosed every 4 weeks compared to every

8 weeks [see Clinical Studies (14.1)]. Some patients may need every 4 week (monthly) dosing
after the first 12 weeks (3 months).

13 Macular Edema Following Retinal Vein Occlusion (RVO)

The recommended dose for EYLEA 1s 2 mg (0.05 ml or 50 microliters) admumstered by
intravitreal injection once every 4 weeks (monthly) [see Clinical S

# Util: doc

In [20]:
%%writefile src/util_doc.py

import os
import sys
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

# local imports
from src.util_text import get_lst_text_from_uploaded_file

def get_doc_chunks_from_lst_text(lst_text,path_pdf):
    """Convert list of text to list of langchain Documents.

    Note: we have chunks, not the page documents

    from langchain.docstore.document import Document
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    """
    from langchain.docstore.document import Document
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from pathlib import Path

    if isinstance(lst_text, str):
        # Take a single string as one page
        lst_text = [lst_text]

    # get docs
    page_docs = [Document(page_content=page) for page in lst_text]

    # Add page numbers as metadata
    for i, doc in enumerate(page_docs):
        doc.metadata["page"] = i + 1

    # Split pages into chunks
    doc_chunks = []

    for doc in page_docs:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
            chunk_overlap=0,
        )
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            doc = Document(
                page_content=chunk, metadata={"page": doc.metadata["page"], "chunk": i}
            )
            # Add sources a metadata
            doc.metadata["source"] = f"{doc.metadata['page']}-{doc.metadata['chunk']}"
            doc.metadata["path_name"] = Path(path_pdf).name
            doc.metadata["path_name_full"] = str(path_pdf)
            doc_chunks.append(doc)
    return doc_chunks

def get_all_doc_chunks_from_path_pdfs(path_pdfs,ocr=False):
    r'''Get document chunks for all the pdfs.

    Depends:
    ========
    - get_lst_text_from_uploaded_file
    - get_doc_chunks_from_lst_text

    '''
    all_doc_chunks = []
    for i, path_pdf in enumerate(path_pdfs):
        lst_text =  get_lst_text_from_uploaded_file(path_pdf,ocr=ocr)
        doc_chunks = get_doc_chunks_from_lst_text(lst_text,path_pdf)
        for doc in doc_chunks:
            doc.metadata["pdf_number"] = str(i+1)
            all_doc_chunks.append(doc)
    return all_doc_chunks

Writing src/util_doc.py


In [21]:
from src.util_doc import get_all_doc_chunks_from_path_pdfs

path_pdfs = [path_pdf_ocr]

all_doc_chunks = get_all_doc_chunks_from_path_pdfs(path_pdfs,ocr=True)
doc_chunk = all_doc_chunks[0]

type(doc_chunk)

langchain_core.documents.base.Document

In [22]:
doc_chunk

Document(page_content='y DOSAGE AND ADMINISTRATION\n\n21 Important Injection Instructions\n\nFor ophthalmic intravitreal injection. EYLEA must only be admumstered by a qualified\nphysician.\n\n2.2 — Neovascular (Wet) Age-Related Macular Degeneration (AMD)\n\nThe recommended dose for EYLEA 1s 2 me (0.05 mL or 50 microliters) admuistered by\nintravitreal injection every 4 weeks (monthly) for the first 12 weeks (3 months), followed by\n\n2 me (0.05 mL) via intravitreal injection once every 8 weeks (2 months). Although EYLEA may\nbe dosed as frequently as 2 me every 4 weeks (monthly), additional efficacy was not\ndemonstrated in most patients when EYLEA was dosed every 4 weeks compared to every\n\n8 weeks [see Clinical Studies (14.1)]. Some patients may need every 4 week (monthly) dosing\nafter the first 12 weeks (3 months).\n\n13 Macular Edema Following Retinal Vein Occlusion (RVO)\n\nThe recommended dose for EYLEA 1s 2 mg (0.05 ml or 50 microliters) admumstered by\nintravitreal injection

# Util: LLM

In [23]:
%%writefile src/util_llm.py
from langchain.embeddings import OpenAIEmbeddings,AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

#======================= llm functions
def get_vectorstore_from_doc_chunks(doc_chunks):
    # new versions of openai needs AzureOpenAIEmbeddings
    embeddings = AzureOpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)
    # embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)
    vectorstore = FAISS.from_documents(documents=doc_chunks, embedding=embeddings)
    return vectorstore

def get_conversation_chain(vectorstore, temp, config):
    # llm
    llm = AzureChatOpenAI(
        openai_api_base=config['openai_api_base'],
        openai_api_version=config['openai_api_version'],
        deployment_name=config['deployment_name'],
        model_name=config['model_name'],
        openai_api_key=config['openai_api_key'],
        openai_api_type=config['openai_api_type'],
        temperature=temp)

    # memory
    memory = ConversationBufferMemory(
                memory_key="chat_history",
                input_key='question',
                output_key='answer',
                return_messages=True)

    # conversation chain
    conv_chain = ConversationalRetrievalChain.from_llm(
                llm=llm,
                retriever=vectorstore.as_retriever(),
                memory=memory,
                return_source_documents=True,
                return_generated_question=True,
                #combine_docs_chain_kwargs={"prompt": chat_prompt_template}
        )

    return conv_chain

Writing src/util_llm.py


In [24]:
from src.util_llm import get_vectorstore_from_doc_chunks, get_conversation_chain
from src.config import config

vectorstore = get_vectorstore_from_doc_chunks(all_doc_chunks)
temp = 0.0
conv_chain = get_conversation_chain(vectorstore, temp, config)

chat_history = []

query = 'what is eylea used for?'

response = conv_chain({'question': query, 'chat_history': chat_history})
chat_history.append(response['answer'])
print(response['answer'])

C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_community\embeddings\azure_openai.py:113: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://qa.gai.cencora.com/aoai to https://qa.gai.cencora.com/aoai/openai.
  warnings.warn(
C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationW

Eylea is used for the treatment of several eye conditions, including:

1. Neovascular (Wet) Age-Related Macular Degeneration (AMD): Eylea is used to treat wet AMD, which is a condition that causes abnormal blood vessel growth in the retina, leading to vision loss.

2. Diabetic Retinopathy (DR) in Patients with Diabetic Macular Edema (DME): Eylea is used to treat DR in patients with DME, which is a complication of diabetes that affects the blood vessels in the retina and can lead to vision loss.

3. Macular Edema Following Retinal Vein Occlusion (RVO): Eylea is used to treat macular edema, which is swelling in the macula, following RVO, a blockage in the veins that carry blood away from the retina.

It is important to note that Eylea should only be administered by a qualified physician.


In [25]:
response.keys()

dict_keys(['question', 'chat_history', 'answer', 'source_documents', 'generated_question'])

In [26]:
response['question']

'what is eylea used for?'

# Util: HTML Templates

In [27]:
notes = r'''
ai = https://www.sideshow.com/storage/product-images/2171/c-3po_star-wars_square.jpg
human = https://flxt.tmsimg.com/assets/p11759522_i_h9_aa.jpg

ai = https://as2.ftcdn.net/v2/jpg/02/38/04/81/1000_F_238048128_OHsRrwAaqeoGCFdpAX6wNgLmGeI0JWFX.jpg
human = https://as2.ftcdn.net/v2/jpg/01/38/26/53/1000_F_138265357_dvVPtWBz5VQXBC0j1a5acIxp488Z2XjW.jpg

"https://resizing.flixster.com/ocuc8yjm8Fu5UK5Ze8lbdp58m9Y=/300x300/v2/https://as2.ftcdn.net/v2/jpg/01/38/26/53/1000_F_138265357_dvVPtWBz5VQXBC0j1a5acIxp488Z2XjW.jpg">

'''

In [28]:
%%writefile src/html_templates.py
css = '''
<style>
.chat-message {
    padding: 1.5rem;
    border-radius: 0.5rem;
    margin-bottom: 1rem;
    display: flex;
    position: relative; /* Added position relative */
}

.chat-message.user {
    background-color: #2b313e;
}

.chat-message.bot {
    background-color: #475063;
}

.chat-message .avatar {
    width: 20%;
}

.chat-message .avatar img {
    max-width: 78px;
    max-height: 78px;
    border-radius: 50%;
    object-fit: cover;
}

.chat-message .message {
    width: 90%;
    padding: 0 1rem;
    color: #fff;
}

.small-font {
    font-size:16px !important;
    color: grey !important;
}

.css-pxxe24 {
    visibility: hidden;
}

</style>
'''

bot_template = '''
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
<div class="chat-message bot">
    <div class="avatar">
        <img src="https://as2.ftcdn.net/v2/jpg/02/38/04/81/1000_F_238048128_OHsRrwAaqeoGCFdpAX6wNgLmGeI0JWFX.jpg">
    </div>
    <div class="message">{{MSG}}</div>
</div>
'''

user_template = '''
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
<div class="chat-message user">
    <div class="avatar">
        <img src="https://as1.ftcdn.net/v2/jpg/01/18/59/38/1000_F_118593824_CAAfUDVnd5ZwlCLFWkzUQMaSyLX7h33j.jpg">
    </div>
    <div class="message">{{MSG}}</div>
</div>

'''


Writing src/html_templates.py


# Util: prompts

In [29]:
%%writefile src/util_prompts.py
gen_prompt = '''
You are a general assistant AI chatbot here to assist the user based on the PDFs they uploaded,
and the subsequent openAI embeddings.

Please assist the user to the best of your knowledge based on uploads, embeddings and the following user input.
If the answer is not available in the uploaded documents, Just say information not available from the input documents.
Don't make up unnecessary answers. Only answer based on the uploaded documents.

USER INPUT: 
        '''

acc_prompt = '''
            You are a academic assistant AI chatbot here to assist the user based on the academic PDFs they uploaded,
            and the subsequent openAI embeddings. This academic persona allows you to use as much outside academic responses as you can.
            But remember this is an app for academix PDF question. Please respond in as academic a way as possible, with an academix audience in mind
            Please assist the user to the best of your knowledge, with this academic persona
            based on uploads, embeddings and the following user input. USER INPUT: 
        '''


prompt = gen_prompt


Writing src/util_prompts.py


# Util: streamlit

In [30]:
%%writefile src/util_streamlit.py
import streamlit as st
import base64
import os
import glob

# local imports
from src.html_templates import css, bot_template, user_template

def display_conversation(prompt):
    r"""Display Conversation

    Depends:
    ========
    - bot_template
    - user_template

    """
    with st.container():
        for i, this_chat_hist in enumerate(reversed(st.session_state.chat_history)):

            question = this_chat_hist.content[len(prompt):]
            answer = this_chat_hist.content

            if i % 2 == 0: # 0,2,4 etc are Bot answer
                st.markdown(bot_template.replace("{{MSG}}", answer), unsafe_allow_html=True)
            else: # 1,3,5 etc are User questions
                st.markdown(user_template.replace("{{MSG}}", question), unsafe_allow_html=True)

def handle_userinput(user_question, prompt):
    response = st.session_state.conversation({'question': (prompt+user_question)})
    st.session_state.response = response
    st.session_state.chat_history = response['chat_history']
    st.session_state.prompt_and_question = response['question']
    st.session_state.questions.append(st.session_state.prompt_and_question)

    # display the conversation
    with st.spinner('Generating response...'):
        display_conversation(prompt)

Writing src/util_streamlit.py


In [31]:
import pyperclip
pyperclip.copy(os.getcwd())

# Main App

In [32]:
%%writefile app.py
# imports
import streamlit as st
import os
import sys
import glob

import openai
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain.llms import AzureOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# local imports
from src.html_templates import css, bot_template, user_template
from src.util_text import get_lst_text_from_uploaded_file
from src.util_doc import get_doc_chunks_from_lst_text
from src.util_prompts import gen_prompt
from src.util_llm import get_vectorstore_from_doc_chunks, get_conversation_chain
from src.util_streamlit import display_conversation
from src.util_streamlit import handle_userinput
from src.config import config

MODEL = 'gpt-35-turbo-16k'
TEMP = 0.0
prompt = gen_prompt

def init_ses_states():
    session_states = {
        "conversation": None,
        "chat_history": None,
        "questions": None,
        "prompt_and_question": "",
    }
    for state, default_value in session_states.items():
        if state not in st.session_state:
            st.session_state[state] = default_value

def process_docs_st(uploaded_files, TEMP, config):
    # document is processed only once and a conversation chain is created.
    # then, we qa the chain multiple time (we do not process same document multiple times)
    st.session_state["conversation"] = None
    st.session_state["chat_history"] = None
    st.session_state["questions"] = []
    st.session_state["prompt_and_question"] = ""
    st.session_state["response"] = None

    all_doc_chunks = []
    for i, uploaded_file in enumerate(uploaded_files):
        file_name = uploaded_file.name
        lst_text = get_lst_text_from_uploaded_file(uploaded_file,ocr=True,pytesseract_path=None)
        doc_chunks = get_doc_chunks_from_lst_text(lst_text,file_name)

        for doc in doc_chunks:
            doc.metadata["pdf_number"] = str(i+1)
            all_doc_chunks.append(doc)

    vectorstore = get_vectorstore_from_doc_chunks(all_doc_chunks)
    conv_chain = get_conversation_chain(vectorstore, temp=TEMP, config=config)

    st.session_state.conversation = conv_chain
    st.session_state.prompt_and_question = ""
    st.session_state.pdf_processed = True

def sidebar():
    global uploaded_files
    global config

    with st.sidebar:
        st.header('Instructions:')
        st.write('1. Upload the pdf (scanned or regular)')
        st.write('2. Click: Process Files + New Chat')
        st.write('3. Start chatting with the chatbot')

        # processing
        with st.expander("Your Documents", expanded=True):
            uploaded_files = st.file_uploader("Upload your PDFs here", accept_multiple_files=True)
            if st.button("Process Files + New Chat"):
                if uploaded_files:
                    # process uploaded files
                    with st.spinner("Processing"):
                        process_docs_st(uploaded_files, TEMP, config)
                else:
                    st.caption("Please Upload At Least 1 PDF")
                    st.session_state.pdf_processed = False

def main():
    st.set_page_config(page_title="Multi-Document Chat Bot", page_icon=":books:")
    st.write(css, unsafe_allow_html=True)
    global uploaded_files, config

    prompt = gen_prompt

    init_ses_states()
    tabs = st.tabs(["Chatbot"] )

    # tab: deploy
    with tabs[0]:
        st.title(":books: PDF Chatbot")
        st.subheader("Created by: Bhishan Poudel")
        sidebar()

        # pdf is processed
        if st.session_state.get("pdf_processed"):
            with st.form("user_input_form"):
                user_question = st.text_input("Ask a question about your documents:")
                send_button = st.form_submit_button("Send")
            if send_button and user_question:
                handle_userinput(user_question, prompt)

        # pdf is not processed
        if not st.session_state.get("pdf_processed"):
            st.caption("Please Upload Atleast 1 PDF Before Proceeding")

if __name__ == '__main__':
    main()

Writing app.py


# Dev

In [39]:
import pyperclip
pyperclip.copy(os.getcwd())

In [37]:
# import subprocess
# git_bash_path = "C:/Users/a126291/AppData/Local/Programs/Git/git-bash.exe"
# _ = subprocess.Popen(['start', git_bash_path, '--cd-to-home'], shell=True)

In [38]:
# import subprocess
# directory_path = r"C:\Users\a126291\OneDrive - AmerisourceBergen(ABC)\data\pdf_files\fda_drugs"
# subprocess.run(['explorer.exe', directory_path])

CompletedProcess(args=['explorer.exe', 'C:\\Users\\a126291\\OneDrive - AmerisourceBergen(ABC)\\data\\pdf_files\\fda_drugs'], returncode=1)

In [36]:
# !explorer.exe .